In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [18]:
traindf = pd.read_csv('data/fashion-mnist_train.csv')
testdf = pd.read_csv('data/fashion-mnist_test.csv')

In [19]:
traindf.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
print(traindf.shape)

(60000, 785)


In [21]:
traindata = traindf.values
testdata = testdf.values

In [32]:
Xtrain = traindata[:traindata.shape[0]//2,1:]
Ytrain = traindata[:traindata.shape[0]//2,0]
Xtest = testdata[:testdata.shape[0]//10,1:]
Ytest = testdata[:testdata.shape[0]//10,0]

In [63]:
onehot_encoder1 = OneHotEncoder(sparse=False)
onehot_encoder2 = OneHotEncoder(sparse=False)

Ytrain_onehot = onehot_encoder1.fit_transform(Ytrain.reshape(-1, 1))
Ytest_onehot = onehot_encoder2.fit_transform(Ytest.reshape(-1, 1))

In [64]:
Ytrain_onehot.shape, Ytest_onehot.shape

((30000, 10), (1000, 10))

In [33]:
print(Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape)

(30000, 784) (30000,) (1000, 784) (1000,)


In [34]:
Xtrain = Xtrain.reshape(-1, 28, 28,1)
Xtest = Xtest.reshape(-1, 28, 28, 1)

In [36]:
print(Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape)

(30000, 28, 28, 1) (30000,) (1000, 28, 28, 1) (1000,)


In [50]:
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [66]:
input_frame = Input(shape=Xtrain[0].shape)
x = Conv2D(8, (5, 5), strides=(2,2), padding="same", activation='relu')(input_frame)
x = Conv2D(16, (3, 3), strides=(2,2), padding="same", activation='relu')(x)
x = Conv2D(32, (3, 3), strides=(2,2), padding="same", activation='relu')(x)
x = Conv2D(32, (3, 3), strides=(2,2), padding="same", activation='relu')(x)
x = AveragePooling2D((2,2))(x)
x = Flatten()(x)
output  = Dense(10, activation='softmax')(x)

model = Model(input_frame, output)


In [67]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [68]:
history = model.fit(Xtrain, Ytrain_onehot, validation_data=(Xtest, Ytest_onehot), batch_size=32, epochs=10, verbose=1)

Epoch 1/10
938/938 [==============================] - 8s 8ms/step - loss: 0.6265 - accuracy: 0.7836 - val_loss: 0.4065 - val_accuracy: 0.8670
Epoch 2/10
938/938 [==============================] - 6s 6ms/step - loss: 0.4111 - accuracy: 0.8544 - val_loss: 0.3595 - val_accuracy: 0.8760
Epoch 3/10
938/938 [==============================] - 6s 6ms/step - loss: 0.3635 - accuracy: 0.8699 - val_loss: 0.3577 - val_accuracy: 0.8730
Epoch 4/10
938/938 [==============================] - 6s 7ms/step - loss: 0.3334 - accuracy: 0.8780 - val_loss: 0.3528 - val_accuracy: 0.8750
Epoch 5/10
938/938 [==============================] - 6s 6ms/step - loss: 0.3114 - accuracy: 0.8866 - val_loss: 0.3811 - val_accuracy: 0.8610
Epoch 6/10
938/938 [==============================] - 6s 6ms/step - loss: 0.2888 - accuracy: 0.8936 - val_loss: 0.3417 - val_accuracy: 0.8780
Epoch 7/10
938/938 [==============================] - 6s 7ms/step - loss: 0.2696 - accuracy: 0.9007 - val_loss: 0.3452 - val_accuracy: 0.8660
Epoch 